## Практика 4: тестування гіпотез про коваріаційну матрицю

In [166]:
import numpy as np
import scipy.stats
#from scipy.stats import chi2
from scipy.stats import mstats
#from scipy.stats import f

import pandas as pd
import pingouin as pg

### Приклад 1. (Example 7.2.2  (Table 3.5)  Rencher. Тести для ков. матриць) 

Зробимо тест на діагональність  матриці ков. матриці (тест Маулчі). 
Нульова гіпотеза $H_0$:  $\Sigma = \sigma^2 I$.

In [2]:
Data_3_5 = np.loadtxt("T3_5_PROBE.DAT", dtype=int)
Data_3_5

array([[ 1, 51, 36, 50, 35, 42],
       [ 2, 27, 20, 26, 17, 27],
       [ 3, 37, 22, 41, 37, 30],
       [ 4, 42, 36, 32, 34, 27],
       [ 5, 27, 18, 33, 14, 29],
       [ 6, 43, 32, 43, 35, 40],
       [ 7, 41, 22, 36, 25, 38],
       [ 8, 38, 21, 31, 20, 16],
       [ 9, 36, 23, 27, 25, 28],
       [10, 26, 31, 31, 32, 36],
       [11, 29, 20, 25, 26, 25]])

In [4]:
Data_3_5 = Data_3_5[:,1:]  # видаляємо перший стовбчик, оскільки там нумерація

In [5]:
n = len(Data_3_5)  # кількість випробувань
n

11

In [6]:
xbar = np.mean(Data_3_5,axis=0) # порахуємо середнє
np_xbar = np.round((xbar), 2)       # округимо до 2го знаку
np_xbar

array([25.55, 34.09, 27.27, 30.73])

In [7]:
# cov
S = np.cov(Data_3_5.T)
S

array([[46.07272727, 28.94545455, 40.33636364, 28.36363636],
       [28.94545455, 60.69090909, 37.37272727, 41.12727273],
       [40.33636364, 37.37272727, 62.81818182, 31.68181818],
       [28.36363636, 41.12727273, 31.68181818, 58.21818182]])

In [8]:
det_S= np.linalg.det(S)   # det(cov)
det_S

1320172.2530909083

In [9]:
Tr_S = np.trace(S, offset=0, axis1=0, axis2=1, dtype=None, out=None)   # trace(cov)
Tr_S

227.8

In [10]:
p = len(S)    #  dim(cov)
p

4

Обчислимо статистику  Маулчі 
$$
u = \frac{p^p det(S)}{(TrS)^p}. 
$$

In [13]:
u = (p**p)*det_S/((Tr_S)**p)
u

0.1255034565249533

Обчислимо статистику 
$$
u'=  -\left(n-1-\frac{2p^2 +p+2}{6p}\right)\log u 
$$

In [14]:
til_u = -(n-1-(2*p**2 +p+2)/(6*p)) * np.log(U)  
til_u

17.468134987880443

Знайдемо квантіль розподілу хі квадрат з $\frac{p(p+1)}{2} = 15$  ступенями свободи.

In [15]:
chi2.ppf(0.95, 14, loc=0, scale=1)  

23.684791304840576

Отримали:  23.69 > 17.47, тому приймаємо нульову гіпотезу  $H_0$ про те, що  $\Sigma = \sigma^2 I$.

In [16]:
# "Build in" function 
# pg.sphericity(data) Mauchly test (sphericity, Sigma = sigma^2 I
data = pd.DataFrame(Data_3_5)
pg.sphericity(data)

SpherResults(spher=True, W=0.7765188667848518, chi2=2.2061495165486082, dof=5, pval=0.8209855255221692)

Увага! pg.sphericity(data) обчислює не $u$- статистику і значення  квантілю $\chi^2$!!! 
Це перевірка іншої сферичності! 
(https://github.com/raphaelvallat/pingouin/blob/dcfdc82bbc7f1ba5991b80717a5ca156617443e8/pingouin/distribution.py).
А також Rencher, розділ   7.2.2

## Приклад 2.  Тест на рівність коваріацій у двох вибірках  
(Example 7.3.2  (Table 5.1)  Rencher) 
Див. також (https://pingouin-stats.org/generated/pingouin.box_m.html).

Зчитуємо таблицю: 

In [168]:
Data_5_1 = np.loadtxt("T5_1_PSYCH.DAT", dtype=int)
Data_5_1M = Data_5_1[0:32,1:] 
Data_5_1F = Data_5_1[32:,1:]

In [22]:
np.shape(Data_5_1M)

(32, 4)

Для застосування вбудованої функції трохи перетворимо дані. 

In [23]:
data = pd.DataFrame(Data_5_1[:,1:], columns=['A', 'B', 'C','D'])
data['group'] = [1] * 32 + [2] * 32 
data.head()

,A,B,C,D,group
0,15,17,24,14,1
1,17,15,32,26,1
2,15,14,29,23,1
3,13,12,10,16,1
4,20,17,26,28,1


Тест Бокса: обчислюємо статистику
$$
M:= \frac{|S_1|^{\nu_1/2}|S_2|^{\nu_2/2}\dots |S_k|^{\nu_k/2}}{|S_{pl}|^{\sum_{i=1}^k \nu_i/2}}, 
$$
де   $\nu_i=n_i-1$ та   $\nu_E=\sum_{i=1}^k  \nu_i =\sum_{i=1}^k  n_i-k$. 

Нехай 
$$
c_1:= \left[\sum_{i=1}^k \frac{1}{\nu_i}-\frac{1}{\sum_{i=1}^k \nu_i} \right] \left(\frac{2p^2 +3p-1}{6(p+1)(k-1)}\right). 
$$
Тоді  за умови виконання $H_0$ 
$$
u:=-2(1-c_1) \ln M \sim \chi^2_{\frac{1}{2}(k-1)p(p+1)}. 
$$
У випадку, коли 
$\nu_1=\nu_2=\dots \nu_k =\nu$, 
$$
c_1= \frac{(k+1)(2p^2 +3p-1)}{6k\nu (p+1)}.
$$

Вбудована функція:

In [25]:
pg.box_m(data, dvs=['A', 'B', 'C','D'], group='group')  # 10 = 0.5* (k-1)*p(p+1), k=2, p=4

,Chi2,df,pval,equal_cov
box,13.550751,10.0,0.194487,True


Отже, приймаємо нульову гіпотезу. 

Порахуємо тепер вручну. 

In [28]:
np.shape(Data_5_1M)

(32, 4)

In [44]:
n1 = np.shape(Data_5_1M)[0]-1
n2 = np.shape(Data_5_1F)[0]-1
d = np.shape(Data_5_1M)[1]

In [48]:
Spl = (n1*np.cov(Data_5_1M.T)+ n2*np.cov(Data_5_1F.T))/(n1+n2)
Spl

array([[ 7.16431452,  6.04737903,  5.69304435,  4.70060484],
       [ 6.04737903, 15.89415323,  8.49243952,  5.85584677],
       [ 5.69304435,  8.49243952, 29.35635081, 13.98084677],
       [ 4.70060484,  5.85584677, 13.98084677, 22.32056452]])

In [49]:
Det1 = np.linalg.det(np.cov(Data_5_1M.T))
Det2 = np.linalg.det(np.cov(Data_5_1F.T))
Det = np.linalg.det(Spl)

M = ((Det1)**(n1/2))*((Det2)**(n2/2))/((Det)**(n1/2+n2/2)) # (7.21) Rencher
M

0.0006889791098952688

ОБчислимо тепер константу $c_1$. Маємо 2 групи спостережень, отже, $k=2$. 

In [51]:
n = n1
k = 2
C1=(k+1)*(2*p**2+3*p-1)/(6*k*n * (p+1))
C1
u= -2*(1-C1)*np.log(M) #(7.23) Rencher
u                      # 

13.550751203746621

Отримали таке саме значення, як вище. 

In [233]:
# 0.5* (k-1)*p*(p+1)=10
chi2.ppf(0.95, 10, loc=0, scale=1)   # 13.55< 18.3  Прийняти  H_0

18.307038053275146

Приймаємо $H_0$. 

## Приклад 3. Гіпотеза про блочну матрицю. 

Перевіримо у попедерньому прикладі гімотезу про те, що матриця $S$ має блочну форму. 

Див. також  https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.stats.mstats.f_value_wilks_lambda.html , але там інше зведення до $F$ статистики. 

Нехай є 2 групи спостережень обсяну $n$ та  розмірністю $p\times 1$  та  $q\times 1$. 
Обчислимо Лямбду Уілкса та  $F$-статистику наступним чином: 
$$
\Lambda :=\frac{|S|}{|S_{xx}| |S_{yy}| }=\frac{|E|}{|E+ H|} \sim \Lambda_{p,\nu_H, \nu_E}, 
$$
де 
$$
E:= S_{yy} - S_{yx}S^{-1}_{xx}S_{xy}, \quad H:= S_{yx}S^{-1}_{xx}S_{xy}.
$$
Порахуємо $F$-статистику
$$
F=\frac{1-\Lambda^{1/t}}{\Lambda^{1/t}}  \frac{df_2}{df_1}, 
$$
$$
df_1= p\nu_H, \quad df_2= wt-\frac{1}{2} (p\nu_H-2), 
$$
$$
w=\nu_E +\nu_H -\frac{1}{2} (p+\nu_H +1) , \quad t = \sqrt{\frac{p^2\nu_H^2-4}{p^2+\nu_H^2-5}}. 
$$
У нашому випадку
$$
\nu_H = q, \quad \nu_E = n-1- q. 
$$

In [169]:
nu_H = np.shape(Data_5_1M)[1]
p = nu_H
q = p
n = np.shape(Data_5_1M)[0] 
nu_E = n - 1 - nu_H
w = nu_E + nu_H - 0.5* (p+nu_H+1)
t = np.sqrt(((p*nu_H)**2 -4)/(p**2 +nu_H**2 -5))
df1 = p*nu_H
df2 = w*t - 0.5 * (df1 -2)

In [170]:
S= np.cov(Data_5_1M.T,Data_5_1F.T)

In [171]:
Det1 = np.linalg.det(np.cov(Data_5_1M.T))
Det2 = np.linalg.det(np.cov(Data_5_1F.T))
Det0  = np.linalg.det(np.cov(Data_5_1M.T,Data_5_1F.T))

In [172]:
Lambda = Det0/(Det1 * Det2)
Lambda

0.7904914380766591

Лямбда дуже мала, отже навряд гіпотеза про незалежність справедлива. Обчислимо $F$-статистику. 

In [175]:
F =round(((1- Lambda**(1/t))/(Lambda**(1/t)) ) * df2/df1,4)
F

0.3698

In [174]:
scipy.stats.f.ppf(q=1-0.05, dfn=df1, dfd=df2)

1.7821860549514332

Отже, приймаємо  нульову гіпотезу. 

Як ще можна обчислити Лямбду: 

In [112]:
Date_new = np.hstack((Data_5_1M,Data_5_1F))
np.shape(Date_new)

(32, 8)

In [115]:
Det0  = np.linalg.det(np.cov(Date_new.T))
Det1 = np.linalg.det(np.cov(Data_5_1M.T))
Det2 = np.linalg.det(np.cov(Data_5_1F.T))

In [116]:
Lambda = Det0/(Det1 * Det2)
Lambda

0.7904914380766591